# Part 3 NSGA II

> Code is inspired from:
> 
> https://medium.com/@rossleecooloh/optimization-algorithm-nsga-ii-and-python-package-deap-fca0be6b2ffc
>
> https://github.com/DEAP/deap/blob/master/examples/ga/nsga2.py
>
>  https://github.com/DEAP/deap/blob/master/deap/tools/emo.py

In [483]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
from copy import deepcopy
# from distutils.command.build_scripts import first_line_re
# Import deque for the stack structure, copy for deep copy nodes
from collections import deque
from sklearn.metrics import accuracy_score
# # Encoding categorical features with preserving the missing values in incomplete features
# from sklearn.preprocessing import (KBinsDiscretizer, LabelEncoder,
#                                    OneHotEncoder, OrdinalEncoder,
#                                    StandardScaler)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier
import random
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from deap import algorithms
from deap import base
from deap import benchmarks
from deap.benchmarks.tools import diversity, convergence, hypervolume
from deap import creator
from deap import tools

In [484]:

# define some constants for the genetic algorithm



CONSTANTS_DICT = {
    "POPULATION_SIZE": 100, # number of individuals in each population
    "MAX_GENERATIONS": 100, # number of generations to run the algorithm
    "CROSSOVER_RATE": 0.9, # crossover rate should always be 100%, based on slides
    "MUTATION_RATE": 0.2, # mutation rate
    "CLASSIFIER":KNeighborsClassifier() , # classifier to use
    # "BOUND_LOW": 0.0, # lower bound for the features
    # "BOUND_UP": 1.0, # upper bound for the features
    # "ETA": 20.0, # crowding degree for mutation  and crossover
}


# dataset load

In [485]:
from queue import Empty


class DatasetPart3:
    def __init__(self, df) :
        self.df=df
        # self.df.columns = self.df.columns.str.strip()
        self.X = self.df.iloc[:,:-1]
        self.y = self.df.iloc[:,-1]
        # self.M = self.df.shape[0]  # number of rows
    
    # @classmethod
    # def constructFromFile(cls, filePath):
    #     """Depends on different ds"""
    #     pass

    def getDfWithSelectedFeatures(self, selectedFeatures):
        """No need to avoid FS bias, just based on df"""
        
        # according to the selected features, get the df with selected features
        colone_X = deepcopy(self.X)
        count, index_to_drop = 0, []
        for i in range(len(selectedFeatures)):
            isSelected = selectedFeatures[i] > 0.5 
            if  isSelected:
                index_to_drop.append(i)
                count += 1
        colone_X.drop(colone_X.columns[index_to_drop], axis=1, inplace=True)
        
        # concat the X and y
        returnedDf = colone_X.join(self.y)
        return returnedDf, count
            
        
        
        # NOTE: never use the below code, it will cause the ValueError !!! It spent me too much time to find this fucking stupid bug which is written by me , 
        # I still need to learn more about programming especially the LOW LEVEL knowledge
        
        # returnedDf = pd.DataFrame()
        # selectedCount = 0
        # for i in range(len(selectedFeatures)):
        #     isSelected = True if selectedFeatures[i] > 0.5 else False
        #     if isSelected:
        #         selectedCount += 1
        #         # concat this feature to the returned dataframe
        #         returnedDf = pd.concat([returnedDf,self.df.iloc[:,i]],axis=1)
        # # concat the class column
        # returnedDf = pd.concat([returnedDf, self.y],axis=1)
        # assert returnedDf.empty == False

        # return returnedDf, selectedCount
    
    @staticmethod
    def run_model(df:pd.DataFrame, classifier=CONSTANTS_DICT["CLASSIFIER"]):

        assert df.empty == False
        # pipe = Pipeline([
        #     ('scaler', StandardScaler()),
        #     ('classifier', classifier)
        #                  ])
        # X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1], test_size=0.2, random_state=42)
        
        # pipe.fit(X_train, y_train)
        # return pipe.score(X_test, y_test)
        
        X = df.iloc[:,:-1]
        y = df.iloc[:,-1]
        
        classifier.fit(X, y)
        return classifier.score(X, y)
        # # # # evaluate the model
        # cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=6)
        # # return the error
        # n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        # return np.mean(n_scores)
        
        

class Vehicle(DatasetPart3):
    def __init__(self, df):
        super().__init__(df)
    
    @classmethod
    def constructFromFile(cls, filePath):
        df = pd.read_csv(filePath, header=None, delim_whitespace=True)
        df.columns = [f"f_{i}" for i in range(len(df.columns))]
        df.rename(columns = {f'f_{len(df.columns)-1}':'class'}, inplace = True)
        return cls(df)
    
class MuskClean(DatasetPart3):
    def __init__(self, df):
        super().__init__(df)

    @classmethod
    def constructFromFile(cls, filePath):
        df = pd.read_csv(filePath, header=None)
        # ignore the first 2 columns since they are NOT numerical, so it would be betteer to ignore them 
        df.drop([0,1], axis=1, inplace=True)
        df.columns = [f"f_{i}" for i in range(len(df.columns))]
        df.rename(columns = {f'f_{len(df.columns)-1}':'class'}, inplace = True)
        return cls(df)
    


In [486]:
ds_vehicle = Vehicle.constructFromFile("./vehicle/vehicle.dat")
print(len(ds_vehicle.X.columns))
ds_vehicle.df.info()


18
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846 entries, 0 to 845
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   f_0     846 non-null    int64 
 1   f_1     846 non-null    int64 
 2   f_2     846 non-null    int64 
 3   f_3     846 non-null    int64 
 4   f_4     846 non-null    int64 
 5   f_5     846 non-null    int64 
 6   f_6     846 non-null    int64 
 7   f_7     846 non-null    int64 
 8   f_8     846 non-null    int64 
 9   f_9     846 non-null    int64 
 10  f_10    846 non-null    int64 
 11  f_11    846 non-null    int64 
 12  f_12    846 non-null    int64 
 13  f_13    846 non-null    int64 
 14  f_14    846 non-null    int64 
 15  f_15    846 non-null    int64 
 16  f_16    846 non-null    int64 
 17  f_17    846 non-null    int64 
 18  class   846 non-null    object
dtypes: int64(18), object(1)
memory usage: 125.7+ KB


In [487]:
ds_mushclean = MuskClean.constructFromFile("./musk/clean1.data")
ds_mushclean.df
# # len(ds_mushclean.x.columns)

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_157,f_158,f_159,f_160,f_161,f_162,f_163,f_164,f_165,class
0,42,-198,-109,-75,-117,11,23,-88,-28,-27,...,-74,-129,-120,-38,30,48,-37,6,30,1.0
1,42,-191,-142,-65,-117,55,49,-170,-45,5,...,-302,60,-120,-39,31,48,-37,5,30,1.0
2,42,-191,-142,-75,-117,11,49,-161,-45,-28,...,-73,-127,-120,-38,30,48,-37,5,31,1.0
3,42,-198,-110,-65,-117,55,23,-95,-28,5,...,-302,60,-120,-39,30,48,-37,6,30,1.0
4,42,-198,-102,-75,-117,10,24,-87,-28,-28,...,-73,-127,51,128,144,43,-30,14,26,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,49,-199,-161,29,-95,-86,-48,2,112,-79,...,-246,-209,33,152,134,47,-43,-15,-10,0.0
472,38,-123,-139,30,-117,-88,214,-13,-74,-129,...,-226,-210,20,55,119,79,-28,4,74,0.0
473,43,-102,-20,-101,-116,200,-166,66,-222,-49,...,32,136,-15,143,121,55,-37,-19,-36,0.0
474,39,-58,27,31,-117,-92,85,21,-73,-68,...,-232,-206,13,45,116,79,-28,3,74,0.0


# set up creator

In [488]:
# 2 minimum objectives, so -1,-1
creator.create("MultiObjMin", base.Fitness, weights=(-1.0, -1.0)) 
# Individual should be a list of binary values, i.e. a list of 0s and 1s
creator.create("Individual", list, fitness=creator.MultiObjMin)

/usr/pkg/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'MultiObjMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/pkg/lib/python3.9/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


define wrapper based fitness evaluate function

In [489]:
# def getTransformedDf(df2Transform:pd.DataFrame):
#     """transform the continous features to discontinous. In other words, due to all features are continous, this functions are used to discretise all continous features.

#     KBins is used to discretise the continous features. The number of bins is set to 10. The strategy is set to uniform.
    
#     Tutorial: https://machinelearningmastery.com/discretization-transforms-for-machine-learning/
    
#     Args:
#         df2Transform (pd.DataFrame): df to transform, all features should be continous
        
#     """ 
#     tempDf = deepcopy(df2Transform)
#     tempDf_x = tempDf.iloc[:,:-1]
#     tempDf_y = tempDf.iloc[:,-1]
#     # tempDf_y = LabelEncoder().fit_transform(tempDf_y)
#     # only transform the continous features, ignore Y
#     kbins = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
#     tempDf_x = kbins.fit_transform(tempDf_x)
#     tempDf = pd.concat([pd.DataFrame(tempDf_x),tempDf_y],axis=1)
#     tempDf.columns = [f"f_{i}" for i in range(len(tempDf.columns))]
#     tempDf.rename(columns = {f'f_{len(tempDf.columns)-1}':'class'}, inplace = True)
#     return tempDf

def wrapperFitnessEvaluation(ds:DatasetPart3, individual:creator.Individual, 
                             classifier=CONSTANTS_DICT["CLASSIFIER"]): #KNN by default
    df_selected,selected_count = ds.getDfWithSelectedFeatures(individual)
    # df_selected = getTransformedDf(df_selected)
        
    acc_score = DatasetPart3.run_model(df_selected, classifier)
    obj1 = 1.0-acc_score # classification error
    obj2 = selected_count/len(individual) #ratio of selected features
    assert 0<=obj1<=1
    assert 0<=obj2<=1
    return obj1, obj2

tool box

> https://www.researchgate.net/publication/235707001_DEAP_Evolutionary_algorithms_made_easy

In [490]:
# from deap import dtm
# toolbox is a class contains the operators that we will use in our genetic programming algorithm
# it can be also be used as the container of methods which enables us to add new methods to the toolbox 
def setup_toolbox(ds:DatasetPart3, randSeed:int) -> base.Toolbox:
    toolbox = base.Toolbox()
    # toolbox.register("map",dtm.map)
    
    # for population size, we use the random.randint function to generate a random integer in the range [min, max]
    random.seed(randSeed)
    # register a method to generate random boolean values
    toolbox.register("attr_bool", random.randint, 0, 1)
    # register a method to generate random individuals
    toolbox.register("IndividualCreator", 
                     tools.initRepeat, 
                     creator.Individual, 
                     toolbox.attr_bool, 
                     n=len(ds.X.columns) # feature number, exclude the class column
                    )
    
    # N is not specificied, so need to specify number of individuals to generate within each population when we call it later
    toolbox.register("PopulationCreator", tools.initRepeat, list, toolbox.IndividualCreator) 

    # toolbox.register("select", tools.emo.selTournamentDCD)
    toolbox.register("select", tools.selNSGA2)
    toolbox.register('selectGen1', tools.selTournament, tournsize=2)
    
    
    
    # toolbox.register("mate", tools.cxSimulatedBinaryBounded, low=CONSTANTS_DICT["BOUND_LOW"], up=CONSTANTS_DICT["BOUND_UP"], eta=CONSTANTS_DICT["ETA"])
    # toolbox.register("mutate", tools.mutPolynomialBounded, low=CONSTANTS_DICT["BOUND_LOW"], up=CONSTANTS_DICT["BOUND_UP"], eta=CONSTANTS_DICT["ETA"], indpb=1.0/len(ds.x.columns))
    

    
    toolbox.register("mate", tools.cxTwoPoint)
    # indpb refer to the probability of mutate happening on each gene, it is NOT the same as mutation rate
    toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/len(ds.X.columns)) 
    
    toolbox.register("evaluate", wrapperFitnessEvaluation, ds) # need to pass individual:list
    return toolbox

run NSGA once 

> https://github.dev/DEAP/deap/blob/master/deap/tools/emo.py
> https://github.dev/DEAP/deap/blob/master/examples/ga/nsga2.py

In [491]:
import copy
from select import select
import time

def run_NSGAII(ds:DatasetPart3, randSeed:int, 
                ngen:int=CONSTANTS_DICT["MAX_GENERATIONS"], 
                popSize:int=CONSTANTS_DICT["POPULATION_SIZE"]):
    # stats
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)
    stats.register("mean", np.mean, axis = 0)
    stats.register("std", np.std, axis=0)
    # for record keeping
    logbook = tools.Logbook()    
    logbook.header = "gen", "mean", "std", "min",  "max"
    
    # create toolbox
    random.seed(randSeed)
    toolbox = setup_toolbox(ds, randSeed)
    
    # calculate objectives
    def evaluate_update_fitness_values(p) :
        """Update the fitness values of each individual for the given the population"""
        fitnesses = list(map(toolbox.evaluate, p))
        for ind, fit in zip(p, fitnesses):
            ind.fitness.values = fit
             
    # create the initial population
    pop = toolbox.PopulationCreator(n=popSize)
    evaluate_update_fitness_values(pop)
    for g in range(ngen):
        offspring = algorithms.varAnd(pop, toolbox, 
                                      cxpb=CONSTANTS_DICT["CROSSOVER_RATE"], 
                                      mutpb=CONSTANTS_DICT["MUTATION_RATE"])
        evaluate_update_fitness_values(offspring)
        pop = toolbox.select(pop + offspring, k=len(pop))
        record = stats.compile(pop)
        logbook.record(gen=g,  **record)
        print(logbook.stream)
    
    
    # # fronts = tools.emo.sortNondominated(pop,k=popSize,first_front_only=False)
    # # for idx,front in enumerate(fronts):
    # #     #print(idx,front)
    # #     for ind in front:
    # #         ind.fitness.values = (idx+1),# change fitness to the order of pareto front
    # # offspring = toolbox.selectGen1(pop, len(pop))
    # # # apply mate and mutate only once
    # # offspring = algorithms.varAnd(offspring,toolbox,
    # #                               CONSTANTS_DICT["CROSSOVER_RATE"],
    # #                               CONSTANTS_DICT["MUTATION_RATE"]) 

    # # for gen_counter in range(1,ngen):
    # #     combined_population = pop + offspring
        
    # #     # fitnesses = toolbox.map(toolbox.evaluate, combined_population)
    # #     # # print(f"fitnesses: {fitnesses}")

    # #     # for ind, fit in zip(combined_population, fitnesses):
    # #     #     ind.fitness.values = fit
    # #     evaluate_fitness_values(combined_population)
        
    # #     # stats
    # #     record = stats.compile(combined_population)
    # #     logbook.record(gen=gen_counter,  **record)
    # #     print(logbook.stream)

        
    # #     fronts = tools.emo.sortNondominated(combined_population,
    # #                                         k=popSize,
    # #                                         first_front_only=False)
        
    # #     for front in fronts:
    # #         tools.emo.assignCrowdingDist(front) # for computing crowding distance
            
    # #     pop = []
    # #     for front in fronts:
    # #         pop += front
    # #     pop = toolbox.clone(pop)
    # #     pop = tools.selNSGA2(pop,k=popSize,nd='standard') # elitism strategy basd on crowded distance

    # #     offspring = toolbox.select(pop,popSize) 
    # #     offspring = toolbox.clone(offspring)
    # #     offspring = algorithms.varAnd(offspring,toolbox,
    # #                               CONSTANTS_DICT["CROSSOVER_RATE"],
    # #                               CONSTANTS_DICT["MUTATION_RATE"]) 
    # # bestInd = tools.selBest(pop,1)[0]
    # # bestFit = bestInd.fitness.values
    # # print("Final population hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))
    # # return pop, logbook, hypervolume(pop, [11.0, 11.0]) # set of non-dominated individuals solutions

    # # This is just to assign the crowding distance to the individuals
    # # no actual selection is done
    # pop = toolbox.select(pop, len(pop))
    # record = stats.compile(pop)
    # logbook.record(gen=0, **record)
    # print(logbook.stream)

    # # Begin the generational process
    # for gen_counter in range(1,ngen):
        
    #     # Vary the pop
    #     offspring = tools.selTournamentDCD(pop, len(pop))
    #     offspring = [toolbox.clone(ind) for ind in offspring]
        
    #     offspring = algorithms.varAnd(offspring, toolbox, 
    #                                   cxpb=CONSTANTS_DICT["CROSSOVER_RATE"], 
    #                                   mutpb=CONSTANTS_DICT["MUTATION_RATE"])
           
    #     # Evaluate all  offsprings individuals 
    #     combined_pop = offspring + pop
    #     evaluate_update_fitness_values(combined_pop)
      
    #     # elitism strategy
    #     # Select the next generation pop
    #     pop = toolbox.select(combined_pop, popSize)
    #     # stats
    #     record = stats.compile(pop)
    #     logbook.record(gen=gen_counter,  **record)
    #     print(logbook.stream)
        
    print("Final pop hypervolume is %f" % hypervolume(pop, [11.0, 11.0]))
    return pop, logbook, hypervolume(pop, [11.0, 11.0]) # set of non-dominated individuals solutions


In [492]:
import matplotlib.pyplot as plt
def run_3_times_with_different_seed(ds:DatasetPart3,
                                     title:str, 
                                     max_gen=CONSTANTS_DICT["MAX_GENERATIONS"],
                                     classifier = CONSTANTS_DICT["CLASSIFIER"],
                                     randSeed = [i+1 for i in range(3)],
                                     run_times=3):
    # run 3 times with different seed
    population_list = []
    logbook_list = []
    hypervolume_list = []
    
    color_list = ['r+','g+','b+','c+','m+','y+','k+']
    for i in range(run_times):
        print('-'*80)
        print('-'*80)
        print(title,"\nRunning GA with seed: ", randSeed[i])
        population, logbook, hypervolume = run_NSGAII(ds, randSeed=randSeed[i], ngen=max_gen, popSize=CONSTANTS_DICT["POPULATION_SIZE"])
        population_list.append(population)
        logbook_list.append(logbook)
        hypervolume_list.append(hypervolume)    
        print('-'*80)
        print('-'*80)
        
        # # plot the result
        front = tools.emo.sortNondominated(population,len(population))[0]
        err_rate = [ind.fitness.values[0] for ind in front]
        ratio_selcted = [ind.fitness.values[1] for ind in front]
        # for ind in front:
        plt.plot(err_rate, ratio_selcted, color_list[i],  ms=2,
                 label = f"seed {randSeed[i]},hypervolume: {hypervolume}")
        plt.ylabel("ratio of selected features")
        plt.xlabel("classification error rate")
        plt.title(f"dataset: {title} \n Objective space")
        plt.tight_layout()
    plt.legend(bbox_to_anchor =(1.3,-0.1), loc='lower center')
    plt.show()
        
        # # fitTuple = [ind.fitness.values for ind in population]
            
        # # plt.plot(fitTuple[0], fitTuple[1], label=f"seed {randSeed[i]}\n hypervolume: {hypervolume}")
        # plt.legend(bbox_to_anchor =(1.3,-0.1), loc='lower center')
        # plt.ylabel("ratio of selected features")
        # plt.xlabel("classification error rate")
        # plt.title(f"dataset: {title} \n Objective space")
    # plt.show()
        
        
    # compare error rates of the obtained solutions with that of using the entire feature set.
    subset_mean_err_rate = [logbook.select("mean")[-1][0] for logbook in logbook_list]
    entire_mean_err_rate = DatasetPart3.run_model(ds.df)
    
    print(f"{title}:\n error rates of the obtained solution: {subset_mean_err_rate}\n error rate of using the entire feature set: {entire_mean_err_rate}")
        
    # print 

    return population_list, logbook_list

In [493]:
ds_vehicle = Vehicle.constructFromFile("./vehicle/vehicle.dat")
pop_list,_ = run_3_times_with_different_seed(ds_vehicle, "vehicle",
                                max_gen=100,
                                run_times=3)
print(f"Solution ")
for i in range(len(pop_list)) :
    pop = pop_list[i]
    print("-"*60)
    print(f"Pop for run with seed {i+1}:")
    for j in range(len(pop)):
        ind = pop[j]
        print(f"Individual: {j}\n\tError rate{ind.fitness.values[0]}\n\t\
          Ratio of selected features: {ind.fitness.values[1]}")
                                
print("-"*80)

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
vehicle 
Running GA with seed:  1
gen	mean                   	std                    	min                    	max                    
0  	[0.21456265 0.43222222]	[0.01833065 0.10703063]	[0.17612293 0.22222222]	[0.27423168 0.66666667]
1  	[0.2058156  0.38166667]	[0.01349205 0.10589274]	[0.17612293 0.16666667]	[0.23995272 0.66666667]
2  	[0.1986643  0.34944444]	[0.01134537 0.10951492]	[0.1678487  0.16666667]	[0.22931442 0.66666667]
3  	[0.19295508 0.33      ]	[0.01072739 0.11542193]	[0.1678487  0.11111111]	[0.22813239 0.55555556]
4  	[0.18957447 0.30388889]	[0.01070478 0.11691904]	[0.1678487 0.       ]  	[0.21394799 0.55555556]
5  	[0.18756501 0.27111111]	[0.01052989 0.09884518]	[0.1678487 0.       ]  	[0.21040189 0.5       ]
6  	[0.18620567 0.25055556]	[0.01232544 0.10900646]	[0.15839243 0.        ]	[0.21040189 0.55555556]
7  	

In [ ]:
ds_mushclean = MuskClean.constructFromFile("./musk/clean1.data")
pop_list,_ =run_3_times_with_different_seed(ds_vehicle, "mushclean",
                                max_gen=100,
                                run_times=3)
print(f"Solution ")
for i in range(len(pop_list)) :
    pop = pop_list[i]
    print("-"*60)
    print(f"Pop for run with seed {i+1}:")
    for j in range(len(pop)):
        ind = pop[j]
        print(f"Individual: {j}\n\tError rate{ind.fitness.values[0]}\n\t\
          Ratio of selected features: {ind.fitness.values[1]}")
                                
print("-"*80)


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
mushclean 
Running GA with seed:  1
gen	mean                   	std                    	min                    	max                    
0  	[0.21520095 0.43      ]	[0.01860178 0.10875843]	[0.17730496 0.16666667]	[0.27423168 0.66666667]
1  	[0.20520095 0.39166667]	[0.01241641 0.10315247]	[0.17730496 0.16666667]	[0.25295508 0.61111111]
2  	[0.19894799 0.34055556]	[0.01247669 0.10112179]	[0.17612293 0.16666667]	[0.23286052 0.55555556]
3  	[0.19338061 0.335     ]	[0.01154768 0.10842731]	[0.17375887 0.16666667]	[0.22695035 0.55555556]
4  	[0.19691489 0.26666667]	[0.01427091 0.08570694]	[0.17375887 0.11111111]	[0.24940898 0.55555556]
5  	[0.19749409 0.23333333]	[0.0112606  0.04714045]	[0.17375887 0.11111111]	[0.21985816 0.38888889]
6  	[0.18996454 0.25      ]	[0.01085966 0.0580017 ]	[0.1713948  0.11111111]	[0.21631206 0.38888889]
7 

KeyboardInterrupt: 